##### QnA chatcot using RAG. context is standford lectures. this is a take home assignment for an application(Ema)

In [39]:
#imports
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.llms import Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


# Data Ingestion

In [ ]:
loader1=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/introduction/",),
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap")))) 
loader2=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/legality/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
loader3=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/modeling/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main_content-wrap"))))
loader4=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/training/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
### we are using 4 lectures: intro, legality, modeling and training. 
### since all the content was inside the main class "main-content-wrap" we used that


In [45]:
loaders=[loader1,loader2,loader3,loader4] #array of loaders to work upon

In [46]:
docs=[]
for loader in loaders:
    docs.extend(loader.load()) #loading documents together

# Transform

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [49]:
split=splitter.split_documents(docs) # splitting documents into chunks 

In [50]:
len(split)

74

In [51]:
split[0].page_content

'LecturesIntroduction \\[\\newcommand{\\sV}{\\mathcal{V}} \\newcommand{\\nl}[1]{\\textsf{#1}} \\newcommand{\\generate}[1]{\\stackrel{#1}{\\rightsquigarrow}}\\]Welcome to CS324! This is a new course on understanding and developing large language models.What is a language model?A brief historyWhy does this course exist?Structure of this course  What is a language model?The classic definition of a language model (LM) is a probability distribution over sequences of tokens. Suppose we have a vocabulary \\(\\sV\\) of a set of tokens. A language model \\(p\\) assigns each sequence of tokens \\(x_1, \\dots, x_L \\in \\sV\\) a probability (a number between 0 and 1):\\[p(x_1, \\dots, x_L).\\]The probability intuitively tells us how “good” a sequence of tokens is. For example, if the vocabulary is \\(\\sV = \\{ \\nl{ate}, \\nl{ball}, \\nl{cheese}, \\nl{mouse}, \\nl{the} \\}\\), the language model might assign (demo):\\[p(\\nl{the}, \\nl{mouse}, \\nl{ate}, \\nl{the}, \\nl{cheese}) = 0.02,\\] \\[p(

# Retriever

In [60]:
system_message=(""" You are a question-answer assistant.
                    Answer the questions based on the context provided or from the chat history.
                    Keep the answers as accurate and concise as possible.
                    If the answer is not present in the context or chat history then say its not available in the context.
                    \n\n
                    {context}.""")

prompt=ChatPromptTemplate.from_messages([("system",system_message),
                                         MessagesPlaceholder("chat_history"),
                                        ("human","{input}")])


In [41]:
model=Ollama(model="llama3")

In [61]:
stuff_doc_chain=create_stuff_documents_chain(model,prompt) #creating stuff document chain or basically chaining llm and prompt

In [ ]:
vector_storage=Chroma.from_documents(split,OllamaEmbeddings(model="nomic-embed-text"), persist_directory="vector_store",collection_name="qna_embeddings") #initializing our vector storage

In [62]:
retriever=vector_store.as_retriever()

In [64]:
#prompt for chat history

system=""" Given a chat history and the latest user question 
    which might reference context in the chat history, 
    formulate a standalone question which can be understood 
    without the chat history. Do NOT answer the question, 
    just reformulate it if needed and otherwise return it as is."""

history_prompt=ChatPromptTemplate.from_messages([("system",system),
                                  MessagesPlaceholder("chat_history"),
                                  ("human","{input}")])


In [65]:
retriever=create_history_aware_retriever(model,retriever,history_prompt)

In [66]:
retriever_chain=create_retrieval_chain(retriever,stuff_doc_chain)

## managing chat history

In [46]:
store={}

def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [67]:
rag_chain=RunnableWithMessageHistory(retriever_chain,get_session_history, input_messages_key="input",
                                     history_messages_key="chat_history", output_messages_key="answer")

In [68]:
config={"configurable":{"session_id":"abcd"}}

In [69]:
response=rag_chain.invoke({"input":"What are some milestone model architectures and papers in the last few years?"},config=config)

In [70]:
print(response["answer"])

Based on the provided context, here are some milestone model architectures and papers in the last few years:

* Recurrent Neural Networks (RNNs), including Long Short Term Memory (LSTMs), allowed the conditional distribution of a token \(x_i\) to depend on the entire context \(x_{1:i-1}\) (effectively \(n = \infty\)), but these were hard to train.
* Transformers are a more recent architecture (developed for machine translation in 2017) that again returned to having fixed context length \(n\), but were much easier to train (and exploited the parallelism of GPUs). Also, \(n\) could be made “large enough” for many applications (GPT-3 used \(n = 2048\)).
* Some notable papers in recent years include:
	+ "Exploring the Limits of Language Modeling" by R. Józefowicz, Oriol Vinyals, M. Schuster, Noam M. Shazeer, and Luke Zettlemoyer (2016)
	+ "Attention is All You Need" by Vaswani et al. (2017)
	+ "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Devlin et a

In [81]:
# response

In [71]:
for docs in response["context"]:
    print(docs.metadata["source"])

https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/


In [75]:
response2=rag_chain.invoke({"input":"what are the types of law?"},config=config)

In [76]:
response2["answer"]

"The provided context mentions that there are different types of law, including:\n\n* Common law (judiciary): Also known as case law, common law is based on judges referencing previous similar cases and making a ruling (precedent).\n* Statutory law (legislature): Also known as written law, statutory law is produced by government agencies through the legislative process (e.g., congress passing a bill).\n* Regulatory law (executive): Also known as administrative law, this is law that is created by the executive branch of government, often focusing on procedures.\n\nPlease let me know if you have any further questions or if there's anything else I can help with!"

In [77]:
for docs in response2["context"]:
    print(docs.metadata["source"])

https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/


In [78]:
reaponse3=rag_chain.invoke({"input":"can you tell me what questions did i ask before"},config=config)

In [79]:
reaponse3["answer"]

'You didn\'t ask any questions before. Your first question was "What are some milestone model architectures and papers in the last few years?". Then, you asked another question about the same topic, and finally, you asked "what are the types of law?"'

# Putting everything at one place

In [21]:
class StanfordLectureBot:
    def __init__(self,vector_store,model="llama3"):
        self.vector_store=vector_store
        self.llm=Ollama(model=model)
        self.system_message=(""" You are a question-answer assistant.
                    Answer the questions based on the context provided.
                    Keep the answers as accurate and concise as possible.
                    If the answer is not present in the context then say its not available in the context.
                    \n\n
                    {context}.""")

        self.prompt=ChatPromptTemplate.from_messages([("system",self.system_message),
                                        ("human","{input}")])
        self.stuff_doc_chain=create_stuff_documents_chain(self.llm,self.prompt)
        self.retriever=self.vector_store.as_retriever()
        self.retriever_chain=create_retrieval_chain(self.retriever,self.stuff_doc_chain)
        
    def howMayIHelpYou(self,query):
        response=self.retriever_chain.invoke({"input":query})
        print(response["answer"])
        print("\n\nSources:\n")
        for docs in response["context"]:
            print(docs.metadata["source"])
    
    def addLecture(self,path):
        loader=WebBaseLoader(web_paths=(path,),
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
        doc=loader.load()
        splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
        split=splitter.split_documents(doc)
        self.retriever.vectorstore.add_documents(split)
        self.retriever_chain=create_retrieval_chain(self.retriever,self.stuff_doc_chain)
        print("Added")
    

In [16]:
#loading vector store
vector_store=Chroma(persist_directory="vector_store",embedding_function=OllamaEmbeddings(model="nomic-embed-text"),collection_name="qna_embeddings")

In [22]:
#creating instance
bot=StanfordLectureBot(vector_store)

In [20]:
#passing query
bot.howMayIHelpYou("What are some milestone model architectures and papers in the last few years?")

Based on the context provided, some milestone model architectures and papers in the last few years include:

1. Transformers (2017): This paper introduced a new architecture for machine translation that replaced recurrent neural networks (RNNs) with self-attention mechanisms. The transformer model was more efficient and accurate than RNNs.

Paper: "Attention Is All You Need" by Ashish Vaswani et al.

2. BERT (2018): Bidirectional Encoder Representations from Transformers (BERT) is a pre-trained language model that uses a multi-layer bidirectional transformer encoder to generate contextualized representations of text.

Paper: "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Jacob Devlin et al.

3. RoBERTa (2019): Robustly Optimized BERT Pretraining Approach (RoBERTa) is an updated version of the BERT model that uses a different optimization approach and adds a few new techniques to improve performance.

Paper: "RoBERTa: A Robustly Optimized BERT Pret

In [23]:
bot.addLecture("https://stanford-cs324.github.io/winter2022/lectures/capabilities/") #capabilities lecture

Added


In [25]:
bot.howMayIHelpYou("what is Penn Tree Bank")

The course exists because the size of neural language models has skyrocketed in recent years due to major hardware advances (e.g., GPUs) and the rise of deep learning in the 2010s. This increase in size has led to emergent behavior, producing new capabilities and societal impact.

Penn Tree Bank is a dataset used for natural language processing tasks, particularly parsing and part-of-speech tagging.


Sources:

https://stanford-cs324.github.io/winter2022/lectures/capabilities/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/legality/
